In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import sys


project_root = Path.cwd()
while not (project_root / "src").exists() and project_root != project_root.parent:
    project_root = project_root.parent

src_path = project_root / "src"
fig_dir = project_root / "figures"
res_dir = project_root / "results"

fig_dir.mkdir(exist_ok=True)
res_dir.mkdir(exist_ok=True)

print("CWD:", Path.cwd())
print("project_root:", project_root)
print("src_path:", src_path, "exists:", src_path.exists())

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from model_proteostasis import ModelParams, simulate_load



CWD: C:\Users\Regi\Documents\Replication-Asymmetry-Model\notebooks
project_root: C:\Users\Regi\Documents\Replication-Asymmetry-Model
src_path: C:\Users\Regi\Documents\Replication-Asymmetry-Model\src exists: True


In [6]:
def mean_misfold_per_step(asym: bool, p: ModelParams) -> float:
    if not asym:
        lam = p.G * p.MU_BASE
    else:
        lam = p.G * (
            p.f_E * (p.MU_LE_E + p.MU_LA_E)
            + p.f_L * (p.MU_LE_L + p.MU_LA_L)
        )
    return p.p_mis * p.Y * lam

def mu_star(asym: bool, d: float, p: ModelParams) -> float:
    mu_mis = mean_misfold_per_step(asym, p)
    return (1 - d) * (mu_mis + p.B) / d


In [4]:
p_base = ModelParams()
d_val = 0.8
T = 500
reps = 1000

B_factors = [0.25, 0.5, 1.0, 2.0, 4.0]
records = []

for f in B_factors:
    p = ModelParams(**{**p_base.__dict__, "B": p_base.B * f})
    
    mu0_analytic = mu_star(False, d=d_val, p=p)
    mu1_analytic = mu_star(True,  d=d_val, p=p)
    delta_analytic = (mu1_analytic - mu0_analytic) / mu0_analytic * 100.0
    
    L0 = simulate_load(False, reps=reps, T=T, d=d_val, gamma=0.0, p=p, seed=123)
    L1 = simulate_load(True,  reps=reps, T=T, d=d_val, gamma=0.0, p=p, seed=456)
    
    mean0_sim = L0.mean()
    mean1_sim = L1.mean()
    delta_sim = (mean1_sim - mean0_sim) / mean0_sim * 100.0
    
    records.append({
        "B_factor": f,
        "B_value": p.B,
        "mu0_analytic": mu0_analytic,
        "mu1_analytic": mu1_analytic,
        "delta_pct_analytic": delta_analytic,
        "mu0_sim": mean0_sim,
        "mu1_sim": mean1_sim,
        "delta_pct_sim": delta_sim,
    })

df_B = pd.DataFrame(records)
df_B


,B_factor,B_value,mu0_analytic,mu1_analytic,delta_pct_analytic,mu0_sim,mu1_sim,delta_pct_sim
0,0.25,500.0,129.5,137.72,6.347490,129.164899,139.610633,8.087131
1,0.50,1000.0,254.5,262.72,3.229862,254.164899,264.610633,4.109826
2,1.00,2000.0,504.5,512.72,1.629336,504.164899,514.610633,2.071888
3,2.00,4000.0,1004.5,1012.72,0.818318,1004.164899,1014.610633,1.040241
4,4.00,8000.0,2004.5,2012.72,0.410077,2004.164899,2014.610633,0.521201


In [5]:
csv_path = res_dir / "B_sensitivity_proteostasis.csv"
df_B.to_csv(csv_path, index=False)
print("Saved:", csv_path)

plt.figure()
plt.plot(df_B["B_factor"], df_B["delta_pct_analytic"], marker="o", label="analytic")
plt.plot(df_B["B_factor"], df_B["delta_pct_sim"], marker="x", linestyle="--", label="simulation")
plt.xlabel("B factor (relative to baseline)")
plt.ylabel("Δ mean load H1 vs H0 [%]")
plt.title("Effect of basal proteotoxic load B on asymmetry contribution")
plt.legend()

fig_path = fig_dir / "B_sensitivity_delta_pct.png"
plt.savefig(fig_path, dpi=300, bbox_inches="tight")
plt.close()
print("Saved figure:", fig_path)


Saved: C:\Users\Regi\Documents\Replication-Asymmetry-Model\results\B_sensitivity_proteostasis.csv
Saved figure: C:\Users\Regi\Documents\Replication-Asymmetry-Model\figures\B_sensitivity_delta_pct.png
